In [ ]:
!brew install mecab mecab-ipadic git curl xz

!sudo apt update -qy && apt install -qy mecab libmecab-dev mecab-ipadic-utf8
!pip install -q konoha[all]
!mkdir data && wget -q https://github.com/himkt/konoha/raw/master/data/model.spm -O data/model.spm
! pip install neologdn

### Load Data

In [1]:
import pandas as pd
from konoha import SentenceTokenizer
from konoha import WordTokenizer
import re
import neologdn
import spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words
import string
import numpy as np

In [2]:
SEED = 13
np.random.seed(SEED)

In [3]:
filepath = "/Users/kapeleshh/PycharmProjects/STS/"
df=  pd.read_csv(filepath + "data/preproc_train.csv")

In [4]:
df['tokmecab'] = df['text']
df['tokjanome'] = df['text']
df['toknagisa'] = df['text']
#df['toksentencepiece'] = df['text']

### Load Data for Japanese stopwords

In [5]:
file1 = open(filepath + "stopwords/Japanese_stopword_list.txt")

# Use this to read file content as a stream:
line = file1.read()
stopJa = line.split()

### Define Tokenizers

In [ ]:
#sentence_tokenizer = SentenceTokenizer()
tokenizers = ["MeCab", "Janome", "nagisa"]
#tokenizers_support_postag = ["MeCab", "Janome", "nagisa"]

word_tokenizers = []
for word_tokenizer_name in tokenizers:
    tokenizer = WordTokenizer(word_tokenizer_name)
    word_tokenizers.append(tokenizer)

#tokenizer = WordTokenizer("Sentencepiece", model_path="./data/model.spm")
#word_tokenizers.append(tokenizer)

# FIXME: `ContextualVersionConflict` happens on Google Colaboratory.
#        If you want to try `Sudachi`, please restart this notebook
#        after running the entire cells once.
#
# ref. https://github.com/nipy/niwidgets/issues/57
#
# tokenizer = WordTokenizer("Sudachi", mode="A", with_postag=True)
# word_tokenizers.append(tokenizer)

print("Finish creating word tokenizers")

### Generate tokens

In [8]:
#df = df[:100000]
cols = ['tokmecab','tokjanome', 'toknagisa']
for wt in range(len(word_tokenizers)):
  for i in range(df.shape[0]):
    #df.iloc[i] = df.iloc[i].to_string()
    df[cols[wt]][i] = neologdn.normalize(df[cols[wt]][i])
    df[cols[wt]][i] = word_tokenizers[wt].tokenize(df[cols[wt]][i])
    df[cols[wt]][i] = [str(r) for r in df[cols[wt]][i]]
    df[cols[wt]][i]= [w for w in df[cols[wt]][i] if(re.findall(r"([a-zA-Z]+)",w) or re.findall(r"([ぁ-んァ-ン]+)",w) or re.findall(r"([一-龯]+)",w) or re.findall(r"(\d+)",w)) ]
    #df[cols[wt]][i]= [w for w in df[cols[wt]][i] if(re.findall(r"([a-zA-Z]+)",w) or re.findall(r"([ぁ-んァ-ン]+)",w) or re.findall(r"([一-龯]+)",w) or re.findall(r"(\d+)",w)) ]
    df[cols[wt]][i] = [word for word in df[cols[wt]][i] if word not in sw_spacy and word not in string.punctuation and word not in stopJa]
 #  df[cols[wt]][i] = [word for word in df[cols[wt]][i] if word.lower() in eng_words or not word.isalpha()]
    df[cols[wt]][i] = ' '.join(df[cols[wt]][i]).split()
    print(cols[wt],i)
  df[cols[wt]] = df[cols[wt]].apply(lambda y: np.nan if len(y)==0 else y)
    #df = df[df[cols[wt]].notna()]
    #result = [str(r) for r in result]
    #print(" ".join(result))

### Write data into pickle and csv

In [ ]:
dfc.to_csv(filepath + 'data/token_compare(full).csv', mode = 'w', index=False)
df.to_pickle(filepath + 'data/pickle/token_compare(full)')